OKX-hsyquant0 Trading Bot

[OKX API Documentation](https://www.okx.com/docs-v5/en/#overview)

In [4]:
import json

f_apikeys = 'apikeys.json'
apikeys = json.load(open(f_apikeys))

In [14]:
flag = "0" # live trading:0 , demo trading：1
api_key = apikeys['apikey']
secret_key = apikeys['secretkey']
passphrase = apikeys['passphrase']

# Check the balance

In [18]:
import okx.Account as Account

accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)

result = accountAPI.get_account_balance()
print(json.dumps(result, indent=2))

body:  
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'eYLY5IPg7KjaexWwgTnvVG6s+lGObMcwUsWHaIdT/Pw=', 'OK-ACCESS-TIMESTAMP': '2024-01-10T02:18:27.486Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/account/balance
{
  "code": "0",
  "data": [
    {
      "adjEq": "",
      "borrowFroz": "",
      "details": [
        {
          "availBal": "2000",
          "availEq": "2000",
          "borrowFroz": "",
          "cashBal": "2000",
          "ccy": "USDT",
          "crossLiab": "",
          "disEq": "2000.48",
          "eq": "2000",
          "eqUsd": "2000.48",
          "fixedBal": "0",
          "frozenBal": "0",
          "interest": "",
          "isoEq": "0",
          "isoLiab": "",
          "isoUpl": "0",
          "liab": "",
          "maxLoan": "",
          "mgnRatio": "",
          "notionalLever": "0",
          "ordFroze

# Check Tradeable Pairs

In [26]:
import okx.PublicData as PublicData

publicDataAPI = PublicData.PublicAPI(flag = flag)

result = publicDataAPI.get_instruments(instType = "FUTURES")
# print(json.dumps(result, indent=2))

'''
The pairs we will be trading is
BTC-USDT and ETH-USDT
'''

header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/public/instruments?instType=FUTURES


'\nThe pairs we will be trading is\nBTC-USDT and ETH-USDT\n'

In [25]:
import okx.Trade as Trade

tradeAPI = Trade.TradeAPI(api_key, secret_key, passphrase, False, flag)

result = accountAPI.set_position_mode(
    posMode="long_short_mode"
)

print(json.dumps(result, indent=2))

body:  {"posMode": "long_short_mode"}
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'k2kEuOd+RQ3H8tCRfljmVALxK6dqNvdnhHUZ4ahvDmA=', 'OK-ACCESS-TIMESTAMP': '2024-01-10T06:01:11.604Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/account/set-position-mode
{
  "code": "0",
  "data": [
    {
      "posMode": "long_short_mode"
    }
  ],
  "msg": ""
}


In [28]:
# limit order
result = tradeAPI.place_order(
    instId="BTC-USDT",
    tdMode="cash",
    side="buy",
    posSide="net",
    ordType="limit",
    px="10",
    sz="0.001"
)
print(result)

if result["code"] == "0":
    print("Successful order request，order_id = ",result["data"][0]["ordId"])
else:
    print("Unsuccessful order request，error_code = ",result["data"][0]["sCode"], ", Error_message = ", result["data"][0]["sMsg"])

body:  {"instId": "BTC-USDT", "tdMode": "cash", "side": "buy", "ordType": "limit", "sz": "0.001", "ccy": "", "clOrdId": "", "tag": "", "posSide": "net", "px": "10", "reduceOnly": "", "tgtCcy": "", "tpTriggerPx": "", "tpOrdPx": "", "slTriggerPx": "", "slOrdPx": "", "tpTriggerPxType": "", "slTriggerPxType": "", "quickMgnType": "", "stpId": "", "stpMode": ""}
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'n7WPx2hQ3x0OjPsYGK3hfg7zoi5fe/L54OfyEp/ml2o=', 'OK-ACCESS-TIMESTAMP': '2024-01-10T06:23:45.171Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/trade/order
{'code': '0', 'data': [{'clOrdId': '', 'ordId': '665199105089871876', 'sCode': '0', 'sMsg': 'Order placed', 'tag': ''}], 'inTime': '1704867825455534', 'msg': '', 'outTime': '1704867825457781'}
Successful order request，order_id =  665199105089871876
